In [1]:
import numpy as np
import gradflow.comp_graph as cg

import autograd as ag

In [2]:
d_in = 7
d_out = 1
d_hid = 13

B = 17 # batch size
w_A_np = np.random.normal(size=(d_in, d_hid))
w_C_np = np.random.normal(size=(d_hid, d_out))
x_np = np.random.normal(size=(B, d_in))
omega_true = np.random.normal(size=(d_in, 1))
def true_fct(z):
    return np.cos(np.dot(z, omega_true))
y_true = np.array([true_fct(z) for z in x_np])

x = cg.Value("x", x_np) # input vector

w_A = cg.Value("w_A", w_A_np) # params in linear layer
linear1 = cg.Dot("linear1")

act1 = cg.Tanh("act1")

w_C = cg.Value("w_C", w_C_np) # params in a linear layer
linear2 = cg.Dot("linear2")

# loss function comput node
loss = cg.MSELoss("mse")

# we use calls to forward pass to define the function topology
x.forward()
w_A.forward()
w_C.forward()
y = linear1.forward(x, w_A)
y = act1.forward(linear1)
y = linear2.forward(act1, w_C)

# finally we pass inputs through a loss function that 
# returns a scalar
l = loss.forward(linear2, y_true)

In [3]:
def test_function(input):
    z = ag.numpy.dot(input, w_A_np)
    z = ag.numpy.tanh(z)
    z = ag.numpy.dot(z, w_C_np)
    l = ag.numpy.mean( (z - y_true)**2. )
    return l, z

In [5]:
atol = 1e-5

l_ag, y_ag = test_function(x_np)
assert np.allclose(y, y_ag, atol=atol)
assert np.allclose(l, l_ag, atol=atol)

In [5]:
my_graph = cg.Graph("my_fun", [loss])
my_graph.backward()

In [7]:
dl_dx_ag = ag.elementwise_grad(test_function)(x_np)

assert np.allclose(x.d_out, dl_dx_ag, atol=atol)

/home/derek/micromamba/envs/blottograd/lib/python3.12/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


In [1]:
### dual numbers test

from gradflow.dual import Dual

In [2]:
a = Dual(2., 3.)
b = Dual(4., 5.)

In [3]:
res = a+b

In [ ]:
res.v

In [ ]:
res.d

In [6]:
res = a*b

In [ ]:
res.v

In [ ]:
res.d

In [ ]:
a-b

In [ ]:
a**3.